<p style="text-align:center;">
    <img src="https://raw.githubusercontent.com/skypilot-org/skypilot/master/docs/source/images/skypilot-wide-light-1k.png" width=500>
</p>

# Running batch inference using LLMs 🤖️

SkyPilot has made running batch inference using LLMs super easy. In this tutorial, we will run offline batch inference on a TODO(skycamp) model on heterogeneous GPUs.

# Learning outcomes 🎯

After completing this notebook, you will be able to:

1. List the GPUs and Accelerators supported by SkyPilot. 
2. Specify different resource types (NVIDIA and AMD GPUs) for your LLM batch inference.
3. Submit a task to run batch inference on any cloud.
4. Use SkyPilot managed jobs to save up to 3x of your cloud costs.

# Specifying resource requirements of tasks

Special resource requirements are specified through the `resources` field in the SkyPilot task YAML. For example, to request 1 L4 GPU for your task, simply add it to the YAML like so:

```yaml
resources:
  accelerators: L4:1

setup: ....

run: .....
```

> **💡 Hint -** In addition to `accelerators`, you can specify many more requirements, such as `disk_size`, a specific `infra`, `instance_type` and more! You can find more details in the [YAML configuration docs](https://skypilot.readthedocs.io/en/latest/reference/yaml-spec.html).

SkyPilot even supports specifying multiple GPUs of different types, across different platforms! For example, to request 1 L4 or 1 MI300, you can do the following to seamlessly switch between them:

```yaml
resources:
  any_of:
    - cloud: kubernetes
      image_id: docker:rocm/vllm:latest
      accelerators: MI300:1
    - image_id: docker:vllm/vllm-openai:latest
      accelerators: L4:1
```

> **💡 Hint -** Notice that different accelerators might need different runtime images.

## <span style="color:green">[DIY]</span> 💻 Launch your LLM batch inference task!

In this tutorial, we will run batch inference on `Qwen/Qwen3-4B-Instruct-2507` model. The model will be run on either 1 L4 GPU from Nvidia or 1 MI300 GPU from AMD. Depending on the availability of the resources, SkyPilot will automatically choose the best option for you.

**Open a terminal and use `sky launch` to create a GPU cluster:**

-------------------------
```console
sky launch -c llm-$SKYPILOT_USER_ID batch_inference.yaml
```
-------------------------

This will take about 2 minutes.

### Expected output

SkyPilot will automatically failover through all locations in Kubernetes and other clouds to find available resources, and you will see output like:

-------------------------
```console
root@c4784b202994:/skycamp-tutorial/02_batch_inference# sky launch -c llm-$SKYPILOT_USER_ID batch_inference.yaml
YAML to run: batch_inference.yaml
Considered resources (1 node):
-----------------------------------------------------------------------------------------------------------
 INFRA                                    INSTANCE         vCPUs   Mem(GB)   GPUS      COST ($)   CHOSEN   
-----------------------------------------------------------------------------------------------------------
 Kubernetes (amd-devclou...8s-skycamp1)   -                16      64        MI300:1   0.00          ✔     
 Kubernetes (amd-devclou...8s-skycamp2)   -                16      64        MI300:1   0.00                
 Kubernetes (amd-devclou...8s-skycamp3)   -                16      64        MI300:1   0.00                
 Kubernetes (amd-devclou...8s-skycamp4)   -                16      64        MI300:1   0.00                
 GCP (us-east4-a)                         g2-standard-16   16      64        L4:1      1.14                
 AWS (us-east-1)                          g6.4xlarge       16      64        L4:1      1.32                
-----------------------------------------------------------------------------------------------------------
Launching a new cluster 'llm-user_2'. Proceed? [Y/n]: 
⚙︎ Launching on Kubernetes.
└── Pod is up.
✓ Cluster launched: llm-user_2.  View logs: sky logs --provision llm-user_2
⚙︎ Syncing files.
Multiple resources are specified for the task, using: Kubernetes(cpus=16, mem=64+, {'MI300': 1}, image_id=docker:rocm/vllm:latest)
⚙︎ Job submitted, ID: 1
├── Waiting for task resources on 1 node.
└── Job started. Streaming logs... (Ctrl-C to exit log streaming; job will not be killed)
(task, pid=3791) INFO 11-20 04:50:24 [__init__.py:225] Automatically detected platform rocm.
(task, pid=3791) INFO 11-20 04:50:27 [run_batch.py:30] vLLM batch processing API version 0.11.1rc2.dev141+g38f225c2a.rocm700
...
(task, pid=3791) INFO 11-20 04:50:27 [run_batch.py:44] Prometheus metrics disabled
(task, pid=3791) INFO 11-20 04:50:34 [model.py:658] Resolved architecture: Qwen3ForCausalLM
(task, pid=3791) INFO 11-20 04:50:34 [model.py:1745] Using max model len 4096
(task, pid=3791) INFO 11-20 04:50:35 [scheduler.py:225] Chunked prefill is enabled with max_num_batched_tokens=2048.
(task, pid=3791) INFO 11-20 04:50:35 [vllm.py:375] Cudagraph is disabled under eager mode
(task, pid=3791) INFO 11-20 04:50:37 [__init__.py:225] Automatically detected platform rocm.
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:50:40 [core.py:730] Waiting for init message from front-end.
...
(task, pid=3791) [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(task, pid=3791) [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(task, pid=3791) [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(task, pid=3791) [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(task, pid=3791) [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(task, pid=3791) [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:50:42 [parallel_state.py:1325] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:50:42 [gpu_model_runner.py:2843] Starting to load model Qwen/Qwen3-4B-Instruct-2507...
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:50:43 [rocm.py:298] Using Rocm Attention backend on V1 engine.
(task, pid=3791) (EngineCore_DP0 pid=9398) WARNING 11-20 04:50:43 [compilation.py:874] Op 'quant_fp8' not present in model, enabling with '+quant_fp8' has no effect
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:50:43 [weight_utils.py:419] Using model weights format ['*.safetensors']
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:50:48 [weight_utils.py:440] Time spent downloading weights for Qwen/Qwen3-4B-Instruct-2507: 5.128488 seconds
Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:01<00:02,  1.45s/it]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:01<00:00,  1.46it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:03<00:00,  1.03s/it]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:03<00:00,  1.01s/it]
(task, pid=3791) (EngineCore_DP0 pid=9398) 
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:50:51 [default_loader.py:314] Loading weights took 3.14 seconds
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:50:51 [gpu_model_runner.py:2904] Model loading took 7.6719 GiB and 8.817031 seconds
(task, pid=3791) (EngineCore_DP0 pid=9398) [aiter] type hints mismatch, override to --> rmsnorm2d_fwd(input: torch.Tensor, weight: torch.Tensor, epsilon: float, use_model_sensitive_rmsnorm: int = 0) -> torch.Tensor
(task, pid=3791) (EngineCore_DP0 pid=9398) [2025-11-20 04:50:52] WARNING core.py:622: type hints mismatch, override to --> rmsnorm2d_fwd(input: torch.Tensor, weight: torch.Tensor, epsilon: float, use_model_sensitive_rmsnorm: int = 0) -> torch.Tensor
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:51:06 [gpu_worker.py:315] Available KV cache memory: 163.45 GiB
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:51:06 [kv_cache_utils.py:1199] GPU KV cache size: 1,190,224 tokens
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:51:06 [kv_cache_utils.py:1204] Maximum concurrency for 4,096 tokens per request: 290.58x
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:51:06 [core.py:243] init engine (profile, create kv cache, warmup model) took 14.75 seconds
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:51:07 [vllm.py:375] Cudagraph is disabled under eager mode
(task, pid=3791) (EngineCore_DP0 pid=9398) INFO 11-20 04:51:07 [gc_utils.py:40] GC Debug Config. enabled:False,top_objects:-1
(task, pid=3791) INFO 11-20 04:51:07 [loggers.py:191] Engine 000: vllm cache_config_info with initialization after num_gpu_blocks is: 74389
(task, pid=3791) INFO 11-20 04:51:07 [run_batch.py:366] Supported tasks: ['generate']
(task, pid=3791) WARNING 11-20 04:51:07 [model.py:1602] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.
(task, pid=3791) INFO 11-20 04:51:07 [serving_chat.py:130] Using default chat sampling params from model: {'temperature': 0.7, 'top_k': 20, 'top_p': 0.8}
(task, pid=3791) INFO 11-20 04:51:07 [run_batch.py:419] Reading batch from https://raw.githubusercontent.com/cblmemo/skycamp25-tutorial/refs/heads/main/02_batch_inference/batch_input.jsonl...
Running batch:   0% Completed | 0/6 [00:00<?, ?req/s]
(task, pid=3791) INFO 11-20 04:51:08 [chat_utils.py:546] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
Running batch: 100% Completed | 6/6 [00:04<00:00,  1.38req/s]
(task, pid=3791) 
(task, pid=3791) INFO 11-20 04:51:11 [run_batch.py:272] Writing outputs to local file results.jsonl
(task, pid=3791) [rank0]:[W1120 04:51:11.741131699 ProcessGroupNCCL.cpp:1522] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())
...
✓ Job finished (status: SUCCEEDED).

📋 Useful Commands
Job ID: 1
├── To cancel the job:          sky cancel llm-user_2 1
├── To stream job logs:         sky logs llm-user_2 1
└── To view job queue:          sky queue llm-user_2
Cluster name: llm-user_2
├── To log into the head VM:    ssh llm-user_2
├── To submit a job:            sky exec llm-user_2 yaml_file
├── To stop the cluster:        sky stop llm-user_2
└── To teardown the cluster:    sky down llm-user_2
root@c4784b202994:/skycamp-tutorial/02_batch_inference#
```
-------------------------


## <span style="color:green">[DIY]</span> 💻 Terminate your batch inference cluster!
Don't forget to terminate it! Use `sky down` to terminate a cluster.

**Run `sky down` to terminate the cluster**

-------------------------
```console
sky down llm-$SKYPILOT_USER_ID
```
-------------------------

## <span style="color:green">[DIY]</span> 💻 Save the cost by 3x with managed spot job!

To use managed spot to use your model with a 3x cost reduction, simply switch the job launch command to `sky jobs launch --use-spot`:

-------------------------
```console
sky jobs launch --use-spot batch_inference.yaml -n batch-inference
```
-------------------------

SkyPilot will automatically recover the job whenever preemption happens. Since our task is periodically checkpointed to the cloud bucket, the recovery will only experience limited progress loss.

### Expected output

You will see a similar output as before, but with a 3x cost reduction!

```console
root@c4784b202994:/skycamp-tutorial/02_batch_inference# sky jobs launch --use-spot batch_inference.yaml -n batch-inference
YAML to run: batch_inference.yaml
Managed job 'batch-inference' will be launched on (estimated):
No resource satisfying Kubernetes([Spot], cpus=16, mem=64+, {'MI300': 1}, image_id=docker:rocm/vllm:latest) on Kubernetes.
- Try specifying a different CPU count, or add "+" to the end of the CPU count to allow for larger instances.
- Try specifying a different memory size, or add "+" to the end of the memory size to allow for larger instances.
Kubernetes: The following features are not supported by Kubernetes:
        Feature        Reason                                           
        spot_instance  Spot instances are not supported in Kubernetes.  
Considered resources (1 node):
-----------------------------------------------------------------------------------------------
 INFRA                     INSTANCE               vCPUs   Mem(GB)   GPUS   COST ($)   CHOSEN   
-----------------------------------------------------------------------------------------------
 AWS (ap-northeast-2d)     g6.4xlarge[Spot]       16      64        L4:1   0.25          ✔     
 GCP (asia-northeast3-a)   g2-standard-16[Spot]   16      64        L4:1   0.30                
-----------------------------------------------------------------------------------------------
🔍 Multiple AWS instances satisfy L4:1. The cheapest [spot](gpus=L4:1, g6.4xlarge, ...) is considered among: g6.4xlarge, gr6.4xlarge.
To list more details, run: sky show-gpus L4

Launching a managed job 'batch-inference'. Proceed? [Y/n]: 
Launching managed job 'batch-inference'...
├── Waiting for task resources on 1 node.
└── Job started. Streaming logs... (Ctrl-C to exit log streaming; job will not be killed)
...
✓ Job finished (status: SUCCEEDED).
✓ Managed job finished: 1 (status: SUCCEEDED).

root@c4784b202994:/skycamp-tutorial/02_batch_inference# 
```

> **💡 Hint** - For detailed information on how to develop, train and serve LLMs, please checkout the [examples](https://github.com/skypilot-org/skypilot/tree/master/llm) in SkyPilot repository.

#### 🎉 Congratulations! You have learnt how to run LLMs with SkyPilot!

Feel free to explore more use cases in our [repository](https://github.com/skypilot-org/skypilot), [blog](https://blog.skypilot.co/), and [documentation](https://skypilot.readthedocs.io/en/latest/). 

We’d love to hear from you — join our community on Slack: [slack.skypilot.co](slack.skypilot.co).

#### Quick Survey for Today's Event

We’d love your feedback! Please take a moment to fill out our [quick survey](https://tinyurl.com/skypilot-survey-skycamp25) 🙌